In [ ]:
# pip install alpaca-trade-api xgboost pandas numpy scikit-learn joblib

import pandas as pd
import numpy as np
import time
import os
import joblib
from datetime import datetime, timedelta, timezone
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from alpaca_trade_api.rest import REST, TimeFrame

# --- Config ---
API_KEY = 'PKKX8RURWJJ7RV358WP5'
SECRET_KEY = 'BP8IYuKLGG3kgbkV72YauX0CGdBg8JPgCpEx2fm8'
BASE_URL = 'https://paper-api.alpaca.markets'
SYMBOLS = ['AAPL', 'MSFT', 'TSLA', 'NVDA', 'GOOG', 'AMZN', 'META']
TRADE_QTY = 1
STOP_LOSS_PCT = 0.02  # 2%
LOG_FILE = 'trade_log.csv'

api = REST(API_KEY, SECRET_KEY, BASE_URL)

# --- Market Time Check ---
def is_market_open():
    clock = api.get_clock()
    return clock.is_open

# --- Get Position ---
def get_position(symbol):
    try:
        pos = api.get_position(symbol)
        return int(pos.qty), float(pos.avg_entry_price)
    except:
        return 0, 0.0

# --- Historical Data ---
def get_data(symbol, days=5):
    now = datetime.now(timezone.utc)
    start = now - timedelta(days=days)
    try:
        bars = api.get_bars(symbol, TimeFrame.Minute, start=start.isoformat(), end=now.isoformat()).df
        bars = bars.reset_index()
        return bars
    except Exception as e:
        print(f"{symbol} data error: {e}")
        return pd.DataFrame()

# --- Feature Engineering ---
def add_features(df):
    df['return'] = df['close'].pct_change()
    df['volatility'] = df['return'].rolling(window=5).std()
    df['ma'] = df['close'].rolling(window=5).mean()
    df['ma_diff'] = df['close'] - df['ma']
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    df.dropna(inplace=True)
    return df

# --- Train Model ---
def train_model(df):
    X = df[['return', 'volatility', 'ma_diff']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    return model

# --- Predict Stocks ---
def rank_stocks():
    predictions = []
    for symbol in SYMBOLS:
        df = get_data(symbol)
        if df.empty:
            continue
        df = add_features(df)
        model = train_model(df)
        features = df[['return', 'volatility', 'ma_diff']].iloc[[-1]]
        prob = model.predict_proba(features)[0][1]  # probability price will go up
        predictions.append((symbol, prob, model, df))
    ranked = sorted(predictions, key=lambda x: x[1], reverse=True)
    return ranked[:5]  # Top 5

# --- Trade Logic ---
def trade(symbol, model, df):
    features = df[['return', 'volatility', 'ma_diff']].iloc[[-1]]
    prediction = model.predict(features)[0]
    price = df['close'].iloc[-1]
    qty, avg_price = get_position(symbol)

    # Stop-loss logic
    if qty > 0 and price < avg_price * (1 - STOP_LOSS_PCT):
        try:
            api.submit_order(symbol=symbol, qty=qty, side='sell', type='market', time_in_force='gtc')
            print(f"STOP-LOSS SELL: {symbol} @ {price:.2f}")
            log_trade(symbol, 'stop-loss', price)
            return
        except Exception as e:
            print(f"Stop-loss error for {symbol}: {e}")
            return

    if prediction == 1 and qty == 0:
        try:
            api.submit_order(symbol=symbol, qty=TRADE_QTY, side='buy', type='market', time_in_force='gtc')
            print(f"BUY: {symbol} @ {price:.2f}")
            log_trade(symbol, 'buy', price)
        except Exception as e:
            print(f"Buy error for {symbol}: {e}")
    elif prediction == 0 and qty > 0:
        try:
            api.submit_order(symbol=symbol, qty=qty, side='sell', type='market', time_in_force='gtc')
            print(f"SELL: {symbol} @ {price:.2f}")
            log_trade(symbol, 'sell', price)
        except Exception as e:
            print(f"Sell error for {symbol}: {e}")
    else:
        print(f"HOLD: {symbol} @ {price:.2f}")

# --- Log Trade ---
def log_trade(symbol, action, price):
    now = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
    equity = float(api.get_account().equity)
    cash = float(api.get_account().cash)
    row = {
        'time': now,
        'symbol': symbol,
        'action': action,
        'price': price,
        'equity': equity,
        'cash': cash
    }
    df = pd.DataFrame([row])
    if os.path.exists(LOG_FILE):
        df.to_csv(LOG_FILE, mode='a', header=False, index=False)
    else:
        df.to_csv(LOG_FILE, index=False)

# --- Main Loop ---
print("Starting bot...")
try:
    while is_market_open():
        top_stocks = rank_stocks()
        for symbol, _, model, df in top_stocks:
            trade(symbol, model, df)
        print("Cycle complete. Sleeping 5 mins...\n")
        time.sleep(300)  # 5 minutes
    print("Market closed. Bot stopped.")
except KeyboardInterrupt:
    print("Bot manually stopped.")


In [ ]:
Forget every 5 mins
